In [1]:
#https://duckdb.org/2021/05/14/sql-on-pandas.html
import pandas as pd
import duckdb

mydf = pd.DataFrame({'a' : [1, 2, 3]})
print(duckdb.query("SELECT SUM(a) FROM mydf").to_df())

   sum(a)
0     6.0


In [ ]:
# When you run a query in SQL, DuckDB will look for Python variables whose name 
# matches the table names in your query and automatically start reading your Pandas DataFrames

# DuckDB is capable of running queries in parallel directly on top of a 
# Pandas DataFrame (or on a Parquet/CSV file, or on an Arrow table, …)

In [4]:
from urllib.request import urlretrieve
import os
from time import time
def download_dataset():
    url = "https://github.com/cwida/duckdb-data/releases/download/v1.0/lineitemsf1.snappy.parquet"
    dst = "lineitemsf1.snappy.parquet"
    if not os.path.exists(dst):
        urlretrieve(url, dst)
    return dst

def run_timed_query(title, desc, query, connection):
    print(title)
    print(desc)
    start = time()
    results = connection.execute(query).fetchdf()
    end = time()
    print(f"Query took {end - start} seconds")
    print(results)

download_dataset()


Basic count
This is just a count of the rows to show the size of the data
Query took 0.010539770126342773 seconds
   count_star()
0       6001215


In [9]:
def download_dataset_2():
    url = "https://github.com/cwida/duckdb-data/releases/download/v1.0/orders.parquet"
    dst = "orders.parquet"
    if not os.path.exists(dst):
        urlretrieve(url, dst)
    return dst

download_dataset_2()

'orders.parquet'

In [5]:
connection = duckdb.connect()
run_timed_query(
    "Basic count",
    "This is just a count of the rows to show the size of the data",
    """SELECT count(*) FROM 'lineitemsf1.snappy.parquet';""",
    connection,
)

Basic count
This is just a count of the rows to show the size of the data
Query took 0.005545377731323242 seconds
   count_star()
0       6001215


In [6]:
run_timed_query(
    "Simple aggregation",
    "This query uses projection pushdown",
    """SELECT sum(l_extendedprice * l_discount) AS revenue
    FROM 'lineitemsf1.snappy.parquet';""",
    connection,
)

Simple aggregation
This query uses projection pushdown
Query took 0.18692922592163086 seconds
        revenue
0  1.147509e+10


In [7]:
run_timed_query(
    "Filter pushdown",
    "This query reads very little from the file as it filters on more columns",
    """SELECT sum(l_extendedprice * l_discount) AS revenue
    FROM
        'lineitemsf1.snappy.parquet'
    WHERE
        l_shipdate >= CAST('1994-01-01' AS date)
        AND l_shipdate < CAST('1995-01-01' AS date)
        AND l_discount BETWEEN 0.05
        AND 0.07
        AND l_quantity < 24; """,
    connection,
)

Filter pushdown
This query reads very little from the file as it filters on more columns
Query took 0.3641471862792969 seconds
        revenue
0  1.231411e+08


In [8]:
connection.execute(
    """SELECT * FROM
        'lineitemsf1.snappy.parquet'
    limit 10
    """
).fetchdf()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,155190,7706,1,17,21168.23,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the
1,1,67310,7311,2,36,45983.16,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold
2,1,63700,3701,3,8,13309.60,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,"riously. regular, express dep"
3,1,2132,4633,4,28,28955.64,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de
4,1,24027,1534,5,24,22824.48,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re
5,1,15635,638,6,32,49620.16,0.07,0.02,N,O,1996-01-30,1996-02-07,1996-02-03,DELIVER IN PERSON,MAIL,arefully slyly ex
6,2,106170,1191,1,38,44694.46,0.00,0.05,N,O,1997-01-28,1997-01-14,1997-02-02,TAKE BACK RETURN,RAIL,ven requests. deposits breach a
7,3,4297,1798,1,45,54058.05,0.06,0.00,R,F,1994-02-02,1994-01-04,1994-02-23,NONE,AIR,ongside of the furiously brave acco
8,3,19036,6540,2,49,46796.47,0.10,0.00,R,F,1993-11-09,1993-12-20,1993-11-24,TAKE BACK RETURN,RAIL,unusual accounts. eve
9,3,128449,3474,3,27,39890.88,0.06,0.07,A,F,1994-01-16,1993-11-22,1994-01-23,DELIVER IN PERSON,SHIP,nal foxes wake.


In [11]:
lineitem = duckdb.query(
    "SELECT * FROM 'lineitemsf1.snappy.parquet'"
).to_df()
orders = duckdb.query(
    "SELECT * FROM 'orders.parquet'"
).to_df()

In [12]:
type(lineitem)

pandas.core.frame.DataFrame

In [13]:
connection.execute(
    """SELECT l_returnflag,
       l_linestatus,
       sum(l_extendedprice),
       min(l_extendedprice),
       max(l_extendedprice),
       avg(l_extendedprice)
FROM lineitem lineitem
JOIN orders orders ON (l_orderkey=o_orderkey)
WHERE l_shipdate <= DATE '1998-09-02'
  AND o_orderstatus='O'
GROUP BY l_returnflag,
         l_linestatus"""
).fetchdf()

,l_returnflag,l_linestatus,sum(l_extendedprice),min(l_extendedprice),max(l_extendedprice),avg(l_extendedprice)
0,N,O,1.081147e+11,901.0,104749.5,38250.450307
